In [12]:
%gui qt5
import napari
import cv2
from bonpy import OpenCVMovieData
from pathlib import Path
import numpy as np
from tqdm import tqdm

In [28]:
movie_data = OpenCVMovieData("/Users/vigji/Desktop/134838_latest_calibration/checkerboard_cropped/checkerboard_central.avi")

In [29]:
movie_data.shape

(13905, 620, 600)

In [33]:
30*20*60*1080*1440

55987200000

: 

In [31]:
plt.im
movie_data[0, 10:300, 40:50]

array([[24, 25, 25, ..., 27, 25, 27],
       [18, 18, 19, ..., 23, 20, 22],
       [20, 22, 23, ..., 28, 26, 27],
       ...,
       [19, 18, 18, ..., 30, 27, 23],
       [19, 18, 18, ..., 30, 27, 23],
       [19, 18, 18, ..., 30, 27, 23]], dtype=uint8)

In [30]:
movie_data.dtype

dtype('uint8')

In [13]:
# We assume that all movies in this folder have the same frame size and camera in
# identical positions
MOV_FOLDER = "/Users/vigji/Desktop/134838_latest_calibration"
MOV_FORMAT = "avi"
SAMPLED_FRAMES = 100

folder = Path(MOV_FOLDER)
first_movie_file = next(folder.glob("*.avi"))
movie_data = OpenCVMovieData(first_movie_file)

In [14]:
n_frames = movie_data.metadata.n_frames

# Sample frames
frames_idxs = np.arange(0, n_frames, n_frames // SAMPLED_FRAMES, dtype=int)
avg_frame = movie_data[frames_idxs, :, :].mean(axis=0)

In [22]:
napari_viewer = napari.Viewer()
napari_viewer.add_image(avg_frame, name="Average frame", contrast_limits=[0, 100])

# Add rectangle layers for all views: bottom, mirror_left, mirror_right, mirror_top, mirror_bottom
# The rectangle layers will be used to crop the frames.
# Start from a guess of the rectangle position and size:
# x, y, width, height
corner_sw = (860, 340)
corner_nw = (240, 340)
corner_ne = (240, 940)
corner_se = (860, 940)
def_side = 220

default_rectangles = {
    "central": [corner_nw, corner_ne, corner_se, corner_sw],
    "mirror_top": [(corner_nw[0] - def_side, corner_nw[1]), (corner_ne[0] - def_side, corner_ne[1]), corner_ne, corner_nw],
    "mirror_bottom": [corner_sw, corner_se, (corner_se[0] + def_side, corner_se[1]), (corner_sw[0] + def_side, corner_sw[1])],
    "mirror_left": [(corner_nw[0], corner_nw[1] - def_side), corner_nw, corner_sw, (corner_sw[0], corner_sw[1] - def_side)],
    "mirror_right": [corner_ne, (corner_ne[0], corner_ne[1] + def_side), (corner_se[0], corner_se[1] + def_side), corner_se],
}
default_colors = {
    "central": "red",
    "mirror_top": "blue",
    "mirror_bottom": "green",
    "mirror_left": "yellow",
    "mirror_right": "purple",
}

for view_name, rect in default_rectangles.items():
    napari_viewer.add_shapes(
        data=np.array([rect]),
        shape_type='rectangle',
        edge_color=default_colors[view_name],
        face_color='#ffffff00',
        edge_width=4,
        opacity=1,
        name=view_name
    )
    napari_viewer.layers[view_name].mode = "select"


In [23]:
# Retrieve final adjusted rectangles data for cropping:

final_rectangles = {}
for view_name in default_rectangles.keys():
    final_rectangles[view_name] = napari_viewer.layers[view_name].data[0].copy()
napari_viewer.close()

In [24]:
# From rectangle coordinates get width and height that could fit all side views.
# (considering that left and right rectangles are vertical and top and bottom are horizontal)
# First coordinate is the top left corner, other coordinates are clockwise.
def get_width_height(rect):
    width = rect[1][1] - rect[0][1]
    height = rect[2][0] - rect[1][0]
    return width, height

all_measures = []
for view_name, rect in final_rectangles.items():
    if view_name == "central":
        continue

    w, h = get_width_height(rect)
    to_append = [w, h] if view_name in ["mirror_top", "mirror_bottom"] else [h, w]
    all_measures.append(to_append)
all_measures = np.array(all_measures)

w, h = all_measures.max(axis=0)

# create dictionary with the final rectangles data for all views (x, y, width, height)
final_rectangles_crops = {}
for view_name, rect in final_rectangles.items():
    x, y = rect[0]
    width, height = get_width_height(rect) if view_name == "central" else (w, h)

    final_rectangles_crops[view_name] = (x, y, width, height)


In [25]:
def stretch_contrast_limits(frame, contrast_lims):
    c_range = contrast_lims[1] - contrast_lims[0]
    frame = cv2.convertScaleAbs(frame, alpha=1.0, beta=contrast_lims[0])
    frame = cv2.convertScaleAbs(frame, alpha=255./c_range, 
                                       beta=-contrast_lims[0]*255./c_range)
    return frame

def fix_coords(coords_dict, frame_width, frame_height):
    coords_dict_fixed = {}
    for view, coords in coords_dict.items():
        coords = tuple(map(int, coords))
        coords = tuple(map(max, coords, (0, 0, 0, 0)))
        coords = tuple(map(min, coords, (frame_width, frame_height, frame_width, frame_height)))

        coords_dict_fixed[view] = coords
    return coords_dict_fixed




def crop_and_save_views(input_file, views_coords, contrast_lims=None, output_prefix="crop",
                        codec=None, test_mode=False):
    """
    Crop and save views from an AVI movie.

    Args:
    - input_file: Path to the input AVI movie.
    - output_prefix: Prefix for the output file names.
    - views_coords: A dictionary with the coordinates for each view:
        {
            'central': (x, y, width, height),
            'left': (x, y, width, height),
            'right': (x, y, width, height),
            'top': (x, y, width, height),
            'bottom': (x, y, width, height)
        }
    - contrast_lims: Contrast limits to apply to the cropped views.
    - codec: Codec to use for the output AVI files. If None, the codec of the input file will be used.
             Useful alternative: cv2.VideoWriter_fourcc(*'XVID')
    - test_mode: If True, only 1000 frames will be processed.
    """
    input_file = Path(input_file)
    output_folder = input_file.parent / f"{input_file.stem}_cropped"
    output_folder.mkdir(exist_ok=True)

    transform_funcs_dict = {"mirror_top": lambda x: cv2.rotate(x, cv2.ROTATE_180),
                            "mirror_bottom": lambda x: x,
                            "mirror_left": lambda x: cv2.rotate(x, cv2.ROTATE_90_COUNTERCLOCKWISE),
                            "mirror_right": lambda x: cv2.rotate(x, cv2.ROTATE_90_CLOCKWISE),
                            "central": lambda x: x}
    
    # Open the video file
    cap = cv2.VideoCapture(str(input_file))
    
    # Extract video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    frame_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    n_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # ensure input coords are integers and within the frame:
    views_coords = fix_coords(views_coords, frame_width, frame_height)
    
    # Use same codec as input video file:
    if codec is None:
        codec = int(cap.get(cv2.CAP_PROP_FOURCC))
    
    outputs = {}
    for view in views_coords:
        x, y, w, h = views_coords[view]

        fname = str(output_folder / f"{input_file.stem}_{view}.avi")
        # print(f"Saving {view} to {fname} (h, w): {h}, {w}")
        outputs[view] = cv2.VideoWriter(fname, codec, fps, (w, h))
    
        
    to_process = 1000 if test_mode else n_frames
    for _ in tqdm(range(to_process)):  #n_frames
        ret, frame = cap.read()
        if not ret:
            break

        # Crop each view and write to file
        for view, (y, x, w, h) in views_coords.items():
            if view in ['mirror_left', 'mirror_right']:
                w, h = h, w
            # print(f"View: {view}, x={x}, y={y}, w={w}, h={h} " )
            cropped_view = frame[y:y+h, x:x+w]
            # print(cropped_view.shape)
            if contrast_lims is not None:
                cropped_view = stretch_contrast_limits(cropped_view, contrast_lims)
            cropped_view = transform_funcs_dict[view](cropped_view)
            # print("After", cropped_view.shape)
            outputs[view].write(cropped_view)
        
    # Release everything
    cap.release()
    for output in outputs.values():
        output.release()

In [26]:
for f in folder.glob("*.avi"):
    crop_and_save_views(f, final_rectangles_crops, contrast_lims=[0, 100], output_prefix="cropped")


100%|██████████| 13905/13905 [02:16<00:00, 102.18it/s]


In [27]:
crop_path = Path("/Users/vigji/Desktop/134838_latest_calibration/checkerboard_cropped")
for f in crop_path.glob("*.avi"):
    if "central" in f.name:
        continue
    input_video_path = str(f)
    output_video_path = str(f.parent / (f.stem + "_mirrored.avi"))

    # Open the input video
    cap = cv2.VideoCapture(input_video_path)

    # Get video properties
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = cap.get(cv2.CAP_PROP_FPS)

    # Define the codec and create a VideoWriter object to save the output video
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (frame_width, frame_height))

    # Read and flip each frame
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        # Flip the frame horizontally (flipCode = 1)
        flipped_frame = cv2.flip(frame, 1)

        # Write the flipped frame
        out.write(flipped_frame)

    # Release everything
    cap.release()
    out.release()